# Earthquakes and Self-Organized Criticality

* Self organized criticality is the phenomena that places the earth's temperature and density in the region where earthquakes happen. 
* In this model, friction force decreses as the velocity of the block relative to the bottom plate increses. 
* Force equations:
    * Force on block from neigboring blocks: $F_b = -k_c(x_i - x_{i+1}) - k_c(x_i - x_{i-1})$
    * Force of a leaf spring on block: $F_l = -k_p(x_i - v_0 t)$
    * Friction force due to bottom plate: $F_f = -\frac{F_0 sin(v_i)}{1 + |v_i / v_f|}$
* Overall equation of motion: $m_i \frac{d^2 x_i}{dt^2} = k_c(x_{i+1} + x_{i-1} - 2x_i) + k_p(v_0 t - x_i) + F_f$
    * If we break this up, we have: $\frac{dx_i}{dt} = v_i$ and $m_i \frac{d v_i}{dt} = k_c(x_{i+1} + x_{i-1} - 2x_i) + k_p(v_0 t - x_i) + F_f$

In [4]:
import pylab as plt
import numpy as np
import random
%matplotlib inline

In [2]:
def friction (vi, vf, F0):
    """
    Returs the friction of the bottom plate against blocks moving at a specific velocity
    
    Arguements: vi - initial velovity of the block
                vf - final velocity of the block
                F0 - the static friction force
    
    Returned: The force due to friction
    """
    
    force = -(F0 * np.sin(vi)) / (1 + abs(vi/vf))
    return force

In [12]:
def blockMotion (t, vi, kp, kc, mass, F0, v0, vf, x, i):
    """
    Returns the differential equation equal to dvi/dt
    
    Arguements: kp - spring constant of leaf springs
                kc - spring constant of springs between blocks
                mass - mass of individual block
                F0 - the static friction force
                v0 - initial velocoty of top plate
                vi - the previous velocity of the block
                vf - the current velocity of the block
                x - the array containting x positions
                i - the index of the current block
                t - time
    
    Returned: The differential equation modeling the motion of the individual blocks
    """
    if i == 0:
        springForces = kc*(x[i+1] - x[i]) + kp * (v0 * t - x[i])
    elif i == len(x):
        springForces = kc*(x[i-1] - x[i]) + kp * (v0 * t - x[i])
    else:
        springForces = kc*(x[i+1] + x[i-1] - 2 * x[i]) + kp * (v0 * t - x[i])
    
    if springForces < F0:
        derivative = 0
    else: 
        if i == 0:
            springForces = (kc*(x[i+1] - x[i]) + kp * (v0 * t - x[i]) + friction (vi, vf, F0))
        elif i == len(x):
            springForces = (kc*(x[i-1] - x[i]) + kp * (v0 * t - x[i]) + friction (vi, vf, F0))
        else:
            derivative = (kc*(x[i+1] + x[i-1] - 2 * x[i]) + kp * (v0 * t - x[i]) + friction (vi, vf, F0))
        
    return derivative

In [ ]:
def euler (f, y0, interval, steps, *args):
    """ Solve ODE by Euler method with fixed number of steps.

    Arguements: f - function giving ODE as y'=f(x,y)
                y0 - initial value
                interval - tuple region (a,b) on which to solve ODE
                steps - number of steps
    
    Returned: (x,y) points, as (steps+1)x2 numpy array.
    """
    
    a, b = interval
    h = (b - a) / steps #step size
    y = y0
    
    xPoints = np.arange (a, b, h)
    yPoints = []
    
    for x in xPoints:
        yPoints.append (y)
        y1 = h * f (y, x, *args)
        y += y1
            
    return xPoints, yPoints

In [10]:
N = 25
averageSpacing = 1
variation = 0.001
blockPositions = []

for n in range(0,N):
    blockPositions.append(n * averageSpacing + (random.random() - 0.5) * 2 * variation)
    
